<a href="https://colab.research.google.com/github/fadhil-code/USSB/blob/main/Learning_with_FrozenLakev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q* Learning with FrozenLake 4x4 

In this Notebook, we'll implement an agent <b>that plays FrozenLake.</b>

![alt text](http://simoninithomas.com/drlc/Qlearning/frozenlake4x4.png)

The goal of this game is <b>to go from the starting state (S) to the goal state (G)</b> by walking only on frozen tiles (F) and avoid holes (H). However, the ice is slippery, **so you won't always move in the direction you intend (stochastic environment)**



## Step -1: Install the dependencies on Google Colab

In [ ]:
!pip install numpy
!pip install gym

## Step 0: Import the dependencies 📚
We use 3 libraries:
- `Numpy` for our Qtable
- `OpenAI Gym` for our FrozenLake Environment
- `Random` to generate random numbers

In [ ]:
import numpy as np
import gym
import random

## Step 1: Create the environment 🎮
- Here we'll create the FrozenLake 8x8 environment. 
- OpenAI Gym is a library <b> composed of many environments that we can use to train our agents.</b>
- In our case we choose to use Frozen Lake.

In [ ]:
env = gym.make("FrozenLake-v0")

## Step 2: Create the Q-table and initialize it 🗄️
- Now, we'll create our Q-table, to know how much rows (states) and columns (actions) we need, we need to calculate the action_size and the state_size
- OpenAI Gym provides us a way to do that: `env.action_space.n` and `env.observation_space.n`

In [ ]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [ ]:
# Create our Q table with state_size rows and action_size columns (64x4)
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Step 3: Create the hyperparameters ⚙️
- Here, we'll specify the hyperparameters

In [ ]:
total_episodes = 20000       # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005            # Exponential decay rate for exploration prob

## Step 4: The Q learning algorithm 🧠
- Now we implement the Q learning algorithm:
  ![alt text](http://simoninithomas.com/drlc/Qlearning//qtable_algo.png)


In [ ]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            #print(exp_exp_tradeoff, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)
            
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)
    

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

Score over time: 0.51265
[[0.43973104 0.04327879 0.11855936 0.09746994]
 [0.02558493 0.01566247 0.01089735 0.05427358]
 [0.0117058  0.01273597 0.01414355 0.02130391]
 [0.00771357 0.01521612 0.00638903 0.02216353]
 [0.48516623 0.01601507 0.01287552 0.01674788]
 [0.         0.         0.         0.        ]
 [0.00227064 0.00091401 0.00140963 0.00217731]
 [0.         0.         0.         0.        ]
 [0.02228002 0.00149077 0.05951301 0.62664784]
 [0.04529812 0.68750425 0.02430949 0.02104454]
 [0.84814016 0.00510121 0.02797392 0.00229768]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.13485276 0.10842163 0.66408103 0.07550059]
 [0.34355268 0.90577688 0.33682066 0.31597022]
 [0.         0.         0.         0.        ]]


## Step 5: Use our Q-table to play FrozenLake ! 👾
- After 10 000 episodes, our Q-table can be used as a "cheatsheet" to play FrozenLake"
- By running this cell you can see our agent playing FrozenLake.

In [ ]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 15:
                print("We reached our Goal 🏆")
            else:
                print("We fell into a hole ☠️")
            
            # We print the number of step it took.
            print("Number of steps", step)
            
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 30
****************************************************
EPISODE  1
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 8
****************************************************
EPISODE  2
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 8
****************************************************
EPISODE  3
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 15
****************************************************
EPISODE  4
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 69


In [3]:
import gym
import numpy as np 

def value_iteration(env, max_iterations=100000, lmbda=0.9):
  stateValue = [0 for i in range(env.nS)]
  newStateValue = stateValue.copy()
  for i in range(max_iterations):
    for state in range(env.nS):
      action_values = []      
      for action in range(env.nA):
        state_value = 0
        for i in range(len(env.P[state][action])):
          prob, next_state, reward, done = env.P[state][action][i]
          state_action_value = prob * (reward + lmbda*stateValue[next_state]) #Value Iteration formula
          state_value += state_action_value
        action_values.append(state_value)      #the value of each action
        best_action = np.argmax(np.asarray(action_values))   # choose the action which gives the maximum value
        newStateValue[state] = action_values[best_action]  #update the value of the state
    if i > 1000: 
      if sum(stateValue) - sum(newStateValue) < 1e-04:   # if there is negligible difference break the loop
        break
        print(i)
    else:
      stateValue = newStateValue.copy()
  return stateValue 

def get_policy(env,stateValue, lmbda=0.9):
  policy = [0 for i in range(env.nS)]
  for state in range(env.nS):
    action_values = []
    for action in range(env.nA):
      action_value = 0
      for i in range(len(env.P[state][action])):
        prob, next_state, r, _ = env.P[state][action][i]
        action_value += prob * (r + lmbda * stateValue[next_state])
      action_values.append(action_value)
    best_action = np.argmax(np.asarray(action_values))
    policy[state] = best_action
  return policy 


def get_score(env, policy, episodes=1000):
  misses = 0
  steps_list = []
  for episode in range(episodes):
    observation = env.reset()
    steps=0
    while True:
      
      action = policy[observation]
      #print(action)
      observation, reward, done, _ = env.step(action)
      steps+=1
      if done and reward == 1:
        env.render()

        print('🏆 You have got the Frisbee after {} steps'.format(steps))
        steps_list.append(steps)
        break
      elif done and reward == 0:
        env.render()

        print("☠️ You fell in a hole!")
        misses += 1
        break
  print('----------------------------------------------')
  print('You took an average of {:.0f} steps to get the frisbee'.format(np.mean(steps_list)))
  print('And you fell in the hole {:.2f} % of the times'.format((misses/episodes) * 100))
  print('----------------------------------------------')



env = gym.make('FrozenLake-v0')

stateValues = value_iteration(env, max_iterations=1000)
policy = get_policy(env, stateValues)
get_score(env, policy,episodes=1000)


Streaming output truncated to the last 5000 lines.
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 78 steps
  (Left)
SFFF
FHFH
FFFH
HFFG
☠️ You fell in a hole!
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 46 steps
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 36 steps
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 11 steps
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 37 steps
  (Left)
SFFF
FHFH
FFFH
HFFG
☠️ You fell in a hole!
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 24 steps
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 21 steps
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 18 steps
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 46 steps
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 7 steps
  (Left)
SFFF
FHFH
FFFH
HFFG
☠️ You fell in a hole!
  (Down)
SFFF
FHFH
FFFH
HFFG
🏆 You have got the Frisbee after 6 steps
  (Down)
SFFF
FHFH
FFFH
HFFG

In [9]:
env.P[3]

{0: [(0.3333333333333333, 3, 0.0, False),
  (0.3333333333333333, 2, 0.0, False),
  (0.3333333333333333, 7, 0.0, True)],
 1: [(0.3333333333333333, 2, 0.0, False),
  (0.3333333333333333, 7, 0.0, True),
  (0.3333333333333333, 3, 0.0, False)],
 2: [(0.3333333333333333, 7, 0.0, True),
  (0.3333333333333333, 3, 0.0, False),
  (0.3333333333333333, 3, 0.0, False)],
 3: [(0.3333333333333333, 3, 0.0, False),
  (0.3333333333333333, 3, 0.0, False),
  (0.3333333333333333, 2, 0.0, False)]}